# Phân tích bệnh lý tim mạch bằng phương pháp Shape Similarity Analysis

## Quy trình
1. Đọc và phân tích dữ liệu.
2. Phân chia ra các vùng.
3. Dựng 3D để quan sát bệnh lý.
4. Gán nhãn bệnh lý khi bác sĩ và kỹ thuật viên chẩn đoán hình ảnh đồng ý.
5. Thực hiện train DeepNeural cho phương pháp Shape Similarity Analysis

#### Import các thư viện cần thiết

In [ ]:
import os
from supporters import *
import SimpleITK as sitk
import nibabel as nib
import skimage.transform as skTrans

#### Quản lý tên dữ liệu

In [ ]:
def RenameFiles(folder_path):
    files = os.listdir(folder_path)
    file_numbers = [int(file.split('_')[1]) for file in files]
    file_number_pairs = list(zip(files, file_numbers))
    file_number_pairs.sort(key=lambda x: x[1])

    # Rename the files in the correct order
    for i, (old_name, _) in enumerate(file_number_pairs, start=1):
        new_name = f"ct_{i:04d}_label.nii.gz"
        old_path = os.path.join(folder_path, old_name)
        new_path = os.path.join(folder_path, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed: {old_name} to {new_name}")

folder_TrainData = "../../data/VnRawData/TrainData/"
folder_SegmentationData = "../../data/VnRawData/SegmentationData/"

RenameFiles(folder_TrainData)
RenameFiles(folder_SegmentationData)

#### Phân vùng dữ liệu

In [ ]:
import os

# Set the path where you want to create the folders
base_path = "../../data/VnRawData/SegmentationData/"

# Create empty folders
for i in range(1, 109):
    folder_name = f"ct_{i:04d}_label"
    folder_path = os.path.join(base_path, folder_name)
    
    # Check if the folder doesn't exist, then create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")


In [ ]:
def SaveImage(image, file_path_save):
    converted_array = np.array(image, dtype=np.float32)
    converted_array = np.transpose(converted_array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nib.save(nifti_file, file_path_save)
    print(f"{file_path_save} successfully")

def Seperation(img, file_path_save):
    array = img.flatten()
    unique_values, counts = np.unique(array, return_counts=True)
    
    for index, value in enumerate(unique_values): 
        label_array = np.copy(img) 
        label_array[np.where(label_array != value)] = 0 
        SaveImage(label_array, file_path_save + f'_{index + 1}.nii.gz')

def Segmentation(folder_path):
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        img_raw = sitk.ReadImage(file_path, sitk.sitkFloat32)
        img = sitk.GetArrayFromImage(img_raw)
        
        file_path_save = file[:13] + '/' + file[:13]
        file_path_save = os.path.join(folder_path, file_path_save)
        
        Seperation(img, file_path_save)
        
     
Segmentation(
    folder_path='../../data/VnRawData/SegmentationData/'
)

#### Xử lý dữ liệu

In [ ]:
import os

# Set the path where you want to create the folders
base_path = "../../data/VnRawData/SegmentationData/"

# Create empty folders
for i in range(1, 109):
    folder_name = f"ct_{i:04d}_label_resized"
    folder_path = os.path.join(base_path, folder_name)
    
    # Check if the folder doesn't exist, then create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")


In [ ]:
def ResizeImage(image, x_, y_,z_):
    return np.array(skTrans.resize(image, (x_, y_, z_), order=1, preserve_range=True, anti_aliasing=False), dtype="float16")

def SaveImage(image, file_path_save):
    converted_array = np.array(image, dtype=np.float32)
    converted_array = np.transpose(converted_array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nib.save(nifti_file, file_path_save)

def Preprocessing(folder_path, folder_save):
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        print(file_path)
        img_raw = sitk.ReadImage(file_path, sitk.sitkFloat32)
        img = sitk.GetArrayFromImage(img_raw)
        img_resized = ResizeImage(img, 512, 600, 600)
        file_path_save = os.path.join(folder_save, file)
        SaveImage(img_resized, file_path_save)
        
    
main_folder_path = "../../data/VnRawData/SegmentationData/"
subfolders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f)) and "resized" not in f]
resize_subfolders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f)) and "resized" in f]

for subfolder, resize_subfolder in zip(subfolders, resize_subfolders):
    subfolder_path = os.path.join(main_folder_path, subfolder)
    resize_subfolder_path = os.path.join(main_folder_path, resize_subfolder)
    
    Preprocessing(
        folder_path=subfolder_path,
        folder_save=resize_subfolder_path
    )

In [ ]:
def dice_coefficient(array1, array2):
    intersection = np.sum(array1 * array2)
    union = np.sum(array1) + np.sum(array2)
    return 2.0 * intersection / union if union > 0 else 1.0

def jaccard_index(array1, array2):
    intersection = np.sum(array1 * array2)
    union = np.sum(np.logical_or(array1, array2))
    return intersection / union if union > 0 else 1.0

In [ ]:
def read_img(path):
    img_raw = sitk.ReadImage(path, sitk.sitkFloat32)
    img = sitk.GetArrayFromImage(img_raw)
    return img

In [ ]:
import json
path = "D:/Documents/GitHub/VascuIAR/DeepLearning/Training/Defects Detection/input/data_label.json"
with open(path, 'r', encoding='utf-8') as file:
    data_label = json.load(file)
    

def comparison(class_label, disease, input_img):
    for key, value in data_label.items():
        if key == disease:
            cnt_true=val_true=cnt_false=val_false=0
            for case, val in value.items():
                if case == class_label:
                    continue
                if cnt_false > 18: 
                    break
                if val==True:
                    cnt_true += 1
                    val_true += (dice_coefficient(
                        input_img,
                        read_img(os.path.join(main_path, f"VHSCDD_{case}_label", f"ct_{case}_{class_label}.nii.gz"))
                    ))
                else:
                    cnt_false += 1
                    val_false+=(dice_coefficient(
                        input_img,
                        read_img(os.path.join(main_path, f"VHSCDD_{case}_label", f"ct_{case}_{class_label}.nii.gz"))
                    ))
            return (val_true/cnt_true, val_false/cnt_false)
        
def display(res, disease):
    print(f"Tỉ lệ mắc {disease}: {(res[0]/(res[0] + res[1]))*100}" , f"Tỉ lệ bình thường: {(res[1]/(res[0] + res[1]))*100}")


def detection(class_label, input_img):
    if class_label == 'label_7':
        display(comparison(class_label=class_label, disease='Cung động mạch chủ đôi', input_img=input_img), disease='Cung động mạch chủ đôi')
        display(comparison(class_label=class_label, disease='Phình động mạch', input_img=input_img), disease='Phình động mạch')
        display(comparison(class_label=class_label, disease='Hẹp eo động mạch chủ', input_img=input_img), disease='Hẹp eo động mạch chủ')
        
    elif class_label == 'label_8':
        comparison(class_label=class_label, disease='Phình động mạch', input_img=input_img)
        comparison(class_label=class_label, disease='Vòng thắt động mạch phổi', input_img=input_img)
        comparison(class_label=class_label, disease='Hẹp eo động mạch chủ', input_img=input_img)
    
    elif class_label == 'label_4':
        comparison(class_label=class_label, disease='Bất thường tĩnh mạch phổi trở về tuần hoàn', input_img=input_img)
        
    elif class_label == 'label_12':
        comparison(class_label=class_label, disease='Bất thường động mạch vành', input_img=input_img)
        
    elif class_label == 'label_10':
        comparison(class_label=class_label, disease='Tĩnh mạch chủ kép', input_img=input_img)
        comparison(class_label=class_label, disease='Bất thường tĩnh mạch phổi trở về tuần hoàn', input_img=input_img)
        
def double_detection(class_label, input_img_1, input_img_2):
    if class_label == ('label_7', 'label_8'):
        A = comparison(class_label=class_label[0], disease='Còn ống động mạch', input_img=input_img_1)
        B = comparison(class_label=class_label[1], disease='Còn ống động mạch', input_img=input_img_2)
        display(((A[0] + B[0])/2, (A[1] + B[1])/2), disease='Còn ống động mạch')
        
        A = comparison(class_label=class_label[0], disease='Thân chung động mạch', input_img=input_img_1)
        B = comparison(class_label=class_label[1], disease='Thân chung động mạch', input_img=input_img_2)
        display(((A[0] + B[0])/2, (A[1] + B[1])/2), disease='Thân chung động mạch')
        
    elif class_label == ('label_8', 'label_9'):
        A = comparison(class_label=class_label[0], disease='Thân chung động mạch', input_img=input_img_1)
        B = comparison(class_label=class_label[1], disease='Thân chung động mạch', input_img=input_img_2)
        display(((A[0] + B[0])/2, (A[1] + B[1])/2), disease='Thân chung động mạch')
        
    elif class_label == ('label_2', 'label_3'):
        A = comparison(class_label=class_label[0], disease='Thông liên thất', input_img=input_img_1)
        B = comparison(class_label=class_label[1], disease='Thông liên thất', input_img=input_img_2)
        display(((A[0] + B[0])/2, (A[1] + B[1])/2), disease='Thông liên thất')
        
def triple_detection(class_label, input_img_1, input_img_2, input_img_3):
    if class_label == ('label_3', 'label_8', 'label_9'):
        A = comparison(class_label=class_label[0], disease='Thất phải hai đường ra', input_img=input_img_1)
        B = comparison(class_label=class_label[1], disease='Thất phải hai đường ra', input_img=input_img_2)
        C = comparison(class_label=class_label[2], disease='Thất phải hai đường ra', input_img=input_img_3)
        display(((A[0] + B[0] + C[0])/2, (A[1] + B[1] + C[1])/2), disease='Thất phải hai đường ra')
        
def quad_detection(class_label, input_img_1, input_img_2, input_img_3, input_img_4):
    if class_label == ('label_2', 'class_3', 'label_8', 'label_9'):
        A = comparison(class_label=class_label[0], disease='Đảo gốc động mạch', input_img=input_img_1)
        B = comparison(class_label=class_label[1], disease='Đảo gốc động mạch', input_img=input_img_2)
        C = comparison(class_label=class_label[2], disease='Đảo gốc động mạch', input_img=input_img_3)
        D = comparison(class_label=class_label[3], disease='Đảo gốc động mạch', input_img=input_img_4)
        display(((A[0] + B[0] + C[0])/2, (A[1] + B[1] + C[1])/2), disease='Đảo gốc động mạch')
    
        
# main calling
main_path = "D:/Documents/GitHub/VascuIAR/DeepLearning/data/VnRawData/VHSCDD_sep_labels"
specified_data = 13
input_path_1 = os.path.join(main_path, f"VHSCDD_0{specified_data}_label", f"ct_0{specified_data}_label_3.nii.gz")
input_path_2 = os.path.join(main_path, f"VHSCDD_0{specified_data}_label", f"ct_0{specified_data}_label_8.nii.gz")
input_path_3 = os.path.join(main_path, f"VHSCDD_0{specified_data}_label", f"ct_0{specified_data}_label_9.nii.gz")

input_img_1 = read_img(input_path_1)
input_img_2 = read_img(input_path_2)
input_img_3 = read_img(input_path_3)

class_label = (input_path_1[-14:-7], input_path_2[-14:-7], input_path_3[-14:-7])
triple_detection(class_label, input_img_1=input_img_1, input_img_2=input_img_2, input_img_3=input_img_3)

In [ ]:
'''
0: background
1: vena cava --> Tĩnh mạch chủ kép + Bất thường tĩnh mạch phôi trờ về tuần hoàn
2: auricle --> Không có
3: coronary --> Bất thường động mạch vành
4: left ventricle --> Thông liên thất &
5: right ventricle --> Thông liên thất & + Thất phải hai đường ra &&
6: left atrium --> Bất thường tĩnh mạch phổi trở về tuần hoàn
7: right atrium --> Không có
8: myocardiuum --> Không có
9: descending aorta --> Cung động mạch chủ + Phình động mạch + Hẹp eo động mạch chủ + Thân chung động mạch & + Còn ống động mạch && + Đảo gốc &&&
10: pulmonmary trunk --> Phình động mạch + Vòng thắt động mạch phổi + còn ống động mạch & + Thân chung động mạch & + Đảo gốc &&&
11: ascending aorta --> Phình động mạch + Thân chung động mạch &
'''